In [91]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import zipfile
from sklearn.metrics import mean_squared_error

In [92]:
zip_path = '../00_Data_Bases/df_recommendersystem.zip' 
csv_filename = 'df_recommendersystem.csv'

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(csv_filename) as f:
        df = pd.read_csv(f)

#### Matriz de interacción basada en clusters

In [93]:
cluster_item_matrix = df.groupby('Customer_Type')[
    [col for col in df.columns if 'department' in col]
].sum()

#### Normalización de la matriz

In [94]:
scaler = MinMaxScaler()
cluster_item_matrix_scaled = pd.DataFrame(
    scaler.fit_transform(cluster_item_matrix),
    index=cluster_item_matrix.index,
    columns=cluster_item_matrix.columns
)

Agrupamos los datos por Customer_Type (que refiere a los 4 diferentes clusters).
Sumamos las interacciones (número de productos comprados en cada departamento) para cada cluster.
Normalizamos los valores de la matriz para que estén en un rango de 0 a 1. Esto asegura que las columnas (departamentos) tengan un impacto equilibrado en los cálculos posteriores.

Este paso nos permite convertir el dataset en una matriz donde:

Las filas representan clusters (Customer_Type).
Las columnas representan departamentos.
Los valores son interacciones normalizadas (0 a 1).

#### Similitud entre departamentos

In [95]:
item_similarity = cosine_similarity(cluster_item_matrix_scaled.T)
item_similarity_df = pd.DataFrame(
    item_similarity, 
    index=cluster_item_matrix_scaled.columns, 
    columns=cluster_item_matrix_scaled.columns
)

Calculamos la similitud entre las columnas (departamentos) usando cosine similarity.
Convertimos la salida en un DataFrame llamado item_similarity_df para que sea más fácil de usar.

Cosine Similarity: La similitud coseno mide cuán similares son dos vectores en un espacio multidimensional. En este caso, queremos saber qué departamentos tienen patrones de interacción similares según los clusters.

Con este procedimiento obtenemos una matriz de similitud donde:

Las filas y columnas representan departamentos.
Los valores indican cuán similares son los departamentos (entre 0 y 1).

#### Recomendaciones basadas en el cluster

In [96]:
def recommend_items_for_cluster(cluster, cluster_item_matrix, item_similarity_df, top_n=5):
    cluster_vector = cluster_item_matrix.loc[cluster]
    scores = cluster_vector @ item_similarity_df
    scores = scores.sort_values(ascending=False)
    interacted_items = cluster_vector > 0
    recommendations = scores[~interacted_items].head(top_n)

    return recommendations.index.tolist()

##### Explicación de la función generada:

Input:

cluster: El cluster para el que queremos generar recomendaciones.

cluster_item_matrix: Matriz de interacción de clusters.

item_similarity_df: Matriz de similitud entre departamentos.

top_n: Número de ítems (departamentos) a recomendar.

Cálculo de Puntuaciones:

Multiplicamos el vector del cluster por la matriz de similitud (cluster_vector @ item_similarity_df). Esto genera una puntuación para cada departamento basada en su similitud con los departamentos que el cluster ya ha interactuado.

Ordenar y Filtrar:

Ordenamos los departamentos por relevancia (puntuación). Eliminamos los departamentos con los que el cluster ya ha interactuado.

Recomendaciones:
Seleccionamos los top_n departamentos con las puntuaciones más altas para finalmente obtener una lista de los mejores departamentos recomendados para un cluster específico.

#### Ejemplo de recomendación para un cluster específico

In [97]:
cluster_example = cluster_item_matrix.index[0]
recommendations = recommend_items_for_cluster(cluster_example, cluster_item_matrix_scaled, item_similarity_df)
print(f"Recomendaciones para el cluster {cluster_example}: {recommendations}")

Recomendaciones para el cluster Básicos Frecuentes: ['department_babies', 'department_bakery', 'department_produce', 'department_pets', 'department_personal care']


Seleccionamos un cluster (cluster_example) de la matriz de interacción.

Llamamos a la función recommend_items_for_cluster para obtener las recomendaciones para este cluster, esto genera una lista de los mejores departamentos recomendados para el cluster seleccionado.

### VALIDACION DE MODELO DE RECOMENDACION

#### Se dividen datos en entrenamiento y prueba (a nivel de cluster)

In [98]:
def train_test_split_matrix(matrix, test_size=0.2, random_state=42):
    np.random.seed(random_state)
    train = matrix.copy()
    test = np.zeros_like(matrix)

    for i, row in enumerate(matrix.values):
        non_zero_indices = np.where(row > 0)[0]
        if len(non_zero_indices) == 0:
            continue
        test_indices = np.random.choice(
            non_zero_indices, size=int(len(non_zero_indices) * test_size), replace=False
        )
        train.values[i, test_indices] = 0
        test[i, test_indices] = row[test_indices]

    return pd.DataFrame(train, index=matrix.index, columns=matrix.columns), \
        pd.DataFrame(test, index=matrix.index, columns=matrix.columns)

In [99]:
train_matrix, test_matrix = train_test_split_matrix(cluster_item_matrix_scaled)

#### Resumen de las matrices

In [100]:
print("Matriz de entrenamiento:")
print(train_matrix.to_string(max_rows=10, max_cols=30))

print("\nMatriz de prueba:")
print(test_matrix.to_string(max_rows=10, max_cols=30))

Matriz de entrenamiento:
                    department_babies  department_bakery  department_beverages  department_breakfast  department_bulk  department_canned goods  department_dairy eggs  department_deli  department_dry goods pasta  department_frozen  department_household  department_international  department_meat seafood  department_missing  department_other  department_pantry  department_personal care  department_pets  department_produce  department_snacks
Customer_Type                                                                                                                                                                                                                                                                                                                                                                                                                                            
Básicos Frecuentes           0.000000           0.000000              0.000000             

#### Metricas de Evaluación

In [101]:
def calculate_ndcg(train_matrix, test_matrix, item_similarity_df, top_n=5):
    ndcg_scores = []
    for cluster in test_matrix.index:
        recommendations = recommend_items_for_cluster(
            cluster, train_matrix, item_similarity_df, top_n=top_n
        )
        actual = test_matrix.loc[cluster].sort_values(ascending=False).head(top_n)
        dcg = sum((1 / np.log2(i + 2)) if item in actual.index else 0 
                for i, item in enumerate(recommendations))
        idcg = sum(1 / np.log2(i + 2) for i in range(len(actual)))
        ndcg_scores.append(dcg / idcg if idcg > 0 else 0)
    return np.mean(ndcg_scores)

##### NDCG (Normalized Discounted Cumulative Gain)

NDCG es una métrica muy utilizada en sistemas de recomendación y motores de búsqueda para evaluar la calidad de las recomendaciones o resultados proporcionados. Mide cuan relevantes son las recomendaciones en función de un determinado orden esperado.

#### Función *calculate_ndcg*
Esta función calcula el NDCG (Normalized Discounted Cumulative Gain) para evaluar la calidad de las recomendaciones generadas por el sistema. NDCG mide qué tan relevantes son las recomendaciones y qué tan bien están ordenadas en comparación con el orden ideal.

#### Pasos de la Función:
Iteración por Clusters:
Para cada cluster, genera recomendaciones (top_n) utilizando la matriz de similitud.

Obtención de Relevancia Real:
Extrae los top_n departamentos más relevantes de la matriz de prueba (interacciones reales).

Cálculo de DCG (Discounted Cumulative Gain):
Mide la ganancia acumulada de las recomendaciones, cargando con más peso a los elementos relevantes en las primeras posiciones.

Cálculo de IDCG (Ideal DCG):
Representa la ganancia máxima posible si las recomendaciones estuvieran ordenadas de manera perfecta según las interacciones reales.

Cálculo de NDCG:
Normaliza el DCG dividiéndolo por el IDCG para obtener un valor entre 0 y 1. Un valor más cercano a 1 indica mejores recomendaciones.

Promedio Final:
Devuelve el NDCG promedio para todos los clusters.

#### Drivers para la elección de NDCG como métrica del modelo:
Evalúa Relevancia: Aporta la información de si los departamentos recomendados son útiles para el usuario.
Considera el Orden: Da más peso a los elementos relevantes en las primeras posiciones.
Interpretable: Un NDCG cercano a 1 indica recomendaciones muy relevantes y bien ordenadas.

In [102]:
ndcg_score = calculate_ndcg(train_matrix, test_matrix, item_similarity_df)
print(f"NDCG Score: {ndcg_score:.4f}")

NDCG Score: 0.9016


#### Interpretación del resultado

El resultado del cálculo de NDCG Score es 0.9016, lo cual tiene un significado positivo en el contexto de un sistema de recomendación.

Escala de NDCG:

El NDCG está normalizado entre 0 y 1:
Un valor de 1.0 indica que las recomendaciones son perfectamente relevantes y están ordenadas en el mejor orden posible.
Un valor cercano a 0 indica que las recomendaciones son irrelevantes o están desordenadas.

Resultado:

Un NDCG de 0.9016 significa que el sistema de recomendación está funcionando muy bien, ya que las recomendaciones son muy relevantes y están casi que perfectamente ordenadas.
Este puntaje indica que el modelo genera recomendaciones que están alineadas con las interacciones reales observadas en el conjunto de prueba.

Relevancia y Orden:

La mayoría de los departamentos recomendados están presentes en las interacciones reales.
Los departamentos más relevantes tienden a aparecer en las primeras posiciones de las recomendaciones, lo que es ideal en sistemas de recomendación.

Conclusión

El sistema de recomendación está mostrando un rendimiento excelente según la métrica NDCG. Si bien no es un puntaje perfecto (1.0), el resultado 0.9016 indica que el modelo satisface las expectativas y que sus recomendaciones son útiles y bien ordenadas para los usuarios.